In [1]:
import random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import transformers
print(transformers.__version__)
from transformers.utils import send_example_telemetry
from transformers import TrainingArguments, Trainer, default_data_collator
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoConfig
from datasets import load_dataset, load_metric
from datasets import ClassLabel, Sequence
from sklearn.metrics import accuracy_score, f1_score

4.32.1


In [2]:
data=pd.read_csv('Data/train.csv')
data=data.dropna()
n=len(data)
print(n)
N=list(range(n))
random.shuffle(N)
data=data.iloc[N][0:1000].reset_index(drop=True)
display(data)
print(data.columns.tolist())
print('context')
print(data.iloc[0,0])
print()
print('response')
print(data.iloc[0,1])

3508


,Context,Response
0,I'm obsessing about a terrible breakup. Everyt...,The best way to move on is to give yourself su...
1,I am having a problem with extended family mem...,"Hi, This sounds like a very challenging and up..."
2,"I think I have depression, anxiety, bipolar di...",It can be difficult to get counseling if you d...
3,I don’t love my sister. I would never wish her...,No one can force emotions. Its fine to not l...
4,When my daughter is stressed about a silly thi...,"As parents, it's hard not to have ""freak out"" ..."
...,...,...
995,We kissed and he grabbed my boobs and we excha...,The issue at hand here is that you're betrayal...
996,I started dating a guy I met online. He told m...,Thank you for your question. Trust is a huge t...
997,What are the basic skills a good counselor nee...,1) An awareness of their own incompetence and ...
998,What should I do when we see each other?,I am so sorry this happened. Sharing a part o...


['Context', 'Response']
context
I'm obsessing about a terrible breakup. Everything is a constant reminder.  How do I move on?

response
The best way to move on is to give yourself sufficient time and space away from your ex so you can heal.  That means no checking out your ex on social media platforms and no contact.  In order to move on and diminish the obsessions, help yourself recognize and accept that the relationship is over, and make sure to get sufficient support in processing all your feelings.  Once that takes place, it can be useful to explore and process with a professional the meaning of the relationship and to understand your part in what transpired.Sometimes people obsess because they have difficulties accepting what's already taken place and want things to be different.  The key to moving forward is to be loving toward yourself, to give yourself permission to grieve the loss, and to start cultivating new and healthy habits/patterns.  Reengage in your present life, ask yo

##### Model For ForCausalLM
**roberta-base

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
#from transformers.models.roberta.modeling_roberta import RobertaForQuestionAnswering

In [4]:
class CustomModel(AutoModelForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        self.config.max_position_embeddings = 514  # Set your desired size
        self.max_sequence_length = 514  # Set the same value for consistency
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                inputs_embeds=None, start_positions=None, end_positions=None, output_attentions=None,
                output_hidden_states=None, return_dict=None):

        # Check and truncate input_ids if necessary
        if input_ids is not None and input_ids.size(1) > self.max_sequence_length:
            input_ids = input_ids[:, :self.max_sequence_length]
            
        # Print the sizes of important tensors for debugging
        print("Input_ids size:", input_ids.size())
        print("Attention_mask size:", attention_mask.size() if attention_mask is not None else None)          

        # Continue with the rest of the forward method
        return super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            start_positions=start_positions,
            end_positions=end_positions,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
    
model_name = "roberta-base"
config = AutoConfig.from_pretrained(model_name)
model = CustomModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)  

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


In [5]:
config

RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.32.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [6]:
model

RobertaForCausalLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [7]:
def format_df(dataframe, tokenizer, max_sequence_length=512):
    formatted_data = []
    
    for _, row in dataframe.iterrows():
        input_text = f"{row['Context']} "
        
        if len(input_text) > max_sequence_length:
            input_text = input_text[:max_sequence_length]

        inputs = tokenizer(input_text, return_tensors="pt", max_length=max_sequence_length, truncation=True)    
 
        if 'Response' in row:
            answer_text = row['Response']
            answer_tokens = tokenizer(answer_text, return_tensors="pt")["input_ids"] 
        else:
            answer_text, answer_tokens = None, None

        formatted_data.append({
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
            #'labels': answer_tokens,
        })

    return formatted_data

In [8]:
formatted_df = format_df(data, tokenizer)
print(formatted_df[0])

Token indices sequence length is longer than the specified maximum sequence length for this model (599 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': tensor([[    0,   100,   437, 34668,   154,    59,    10,  6587, 21310,     4,
          9567,    16,    10,  5891,  8306,     4,  1437,  1336,   109,    38,
           517,    15,   116,  1437,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}


#### CustomDataset

In [9]:
class CustomDataset(Dataset):
    def __init__(self, formatted_data):
        self.formatted_data = formatted_data

    def __len__(self):
        return len(self.formatted_data)

    def __getitem__(self, idx):
        return {
            'input_ids': self.formatted_data[idx]['input_ids'].squeeze(),
            'attention_mask': self.formatted_data[idx]['attention_mask'].squeeze(), 
            #'labels': self.formatted_data[idx]['labels'].squeeze() if self.formatted_data[idx]['labels'] is not None else None,
            'start_positions': self.formatted_data[idx]['start_positions'] if 'start_positions' in self.formatted_data[idx] else None,
            'end_positions': self.formatted_data[idx]['end_positions'] if 'end_positions' in self.formatted_data[idx] else None
        }
    
n=len(formatted_df)
trainset = CustomDataset(formatted_df[0:n*4//5])
testset = CustomDataset(formatted_df[n*4//5:])

In [10]:
from torch.nn.utils.rnn import pad_sequence

def custom_collate_fn(batch):
    input_ids = pad_sequence([item['input_ids'] for item in batch], batch_first=True)
    attention_mask = pad_sequence([item['attention_mask'] for item in batch], batch_first=True)

    #labels = pad_sequence([item['labels'] for item in batch if item['labels'] is not None], batch_first=True) if any(item['labels'] is not None for item in batch) else None

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        #'labels': labels,
    }
        
batch_size = 8  

train_dataloader = DataLoader(trainset, batch_size=batch_size, 
                              shuffle=True, collate_fn=custom_collate_fn)
test_dataloader = DataLoader(testset, batch_size=batch_size, 
                              shuffle=False, collate_fn=custom_collate_fn)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
model.to(device)
model.train()

RobertaForCausalLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

#### train

In [12]:
for batch in train_dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    #labels = batch['labels'].to(device) if batch['labels'] is not None else None

    outputs = model(input_ids=input_ids, attention_mask=attention_mask,)# labels=labels

#### Inference

In [13]:
model.eval()
all_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # Get the predicted token IDs
        logits = outputs.logits
        predicted_ids = torch.argmax(logits, dim=-1)

        # Convert token IDs to tokens
        predicted_tokens = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

        # Extend the list of predictions
        all_predictions.extend(predicted_tokens)

In [14]:
## BLUE Score, Accuracy and F1 Score

from nltk.translate.bleu_score import corpus_bleu

test=data[n*4//5:]
test['predicted']=all_predictions
test=test[['Response','predicted']]
#test_df['answer_text']=test_df['answers'].apply(lambda x:x.get('text',[''])[0])
display(test)

references=test['Response'].tolist()
hypotheses =test['predicted'].tolist()

bleu_score = corpus_bleu(references, hypotheses)
#accuracy = accuracy_score(references, hypotheses)
#f1 = f1_score(references, hypotheses, average='micro')

print(f"BLEU Score: {bleu_score:.4f}")
#print(f"Accuracy: {accuracy:.4f}")
#print(f"F1 Score: {f1:.4f}")

C:\Users\shimu\AppData\Local\Temp\ipykernel_26828\2262833351.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted']=all_predictions


,Response,predicted
800,Alot of our choices have to do with what we ha...,I am so terrified of having sex anymore becaus...
801,Wonderful! I am so excited for you. What a h...,I have always wanted to have a transition from...
802,Hello. Being a teenager in our society today c...,I'm in my late teens and live with my dad. Th...
803,"One key factor to consider is, are you able to...",I've only been married three months. Every wee...
804,Your situation sounds extremely frustrating.Yo...,My mother is combative with me when I say I do...
...,...,...
995,The issue at hand here is that you're betrayal...,We kissed and he grabbed my boobs and we excha...
996,Thank you for your question. Trust is a huge t...,I started dating a guy I met online. He told m...
997,1) An awareness of their own incompetence and ...,What are the basic skills a good counselor nee...
998,I am so sorry this happened. Sharing a part o...,What should I do when we see each other?


BLEU Score: 0.0000


C:\Users\shimu\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\shimu\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\shimu\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnin

In [15]:
print('response')
print(test.iloc[0,0])
print()
print('predicted')
print(test.iloc[0,1])

response
Alot of our choices have to do with what we have been taught and partly our fear of trusting ourselves to do what is right. There is so much more going on within you than you realize. We can help you bring it to the surface. Please contact us when you can and in whatever way you can to discuss what's going on with you.

predicted
I am so terrified of having sex anymore because I have been told over and over that sex is dangerous even though me and my partner used both forms of protection. My partner is not happy about this and simply wants more sex, and honestly, I want to give that to her. 
